# Gérer l’encodage des caractères avec une ILC

Le *shell* d’un système tente toujours d’interpréter ce qui est envoyé dans la sortie standard avec le jeu de caractères UTF-8. S’il en est incapable, plutôt que de renvoyer une erreur, il va tenter de substituer un caractère.

Essayons de lire le fichier *ete.txt* depuis un terminal :

In [ ]:
! cat data/ete.utf8.txt

Le mot compte trois caractères mais cinq bytes :

In [ ]:
! cat data/ete.utf8.txt | wc -m && cat data/ete.utf8.txt | wc -c

Une représentation en binaire du fichier permet de révéler les cinq octets :

In [ ]:
! cat data/ete.utf8.txt | xxd -b

Pour ces cinq octets, l’utilitaire `xxd` nous apprend qu’il n’a de représentation possible en ASCII ni pour les deux premiers ni pour les deux derniers.

## Détecter l’encodage d’un fichier

L’utilitaire `file` associé à l’option `-I` permet de déduire l’encodage d’un fichier :

In [ ]:
! file -I data/ete.utf8.txt
! file -I data/ete.latin1.txt

## Modifier l’encodage d’un fichier

Quand le décodage natif du *shell* en UTF-8 ne permet pas d’afficher correctement le message, il devient utile de se servir d’outils pour le décoder correctement. L’utilitaire `iconv` est ainsi prévu pour modifier l’encodage d’un fichier. Le fichier suivant n’est pas lu correctement par le *shell* car encodé en LATIN-1 :

In [ ]:
! cat data/ete.latin1.txt

Avec l’option `-f` on précise depuis quel encodage le traiter :

In [ ]:
! iconv -f LATIN1 data/ete.latin1.txt

Si le décodage avec le jeu de caractères préconisé est impossible, une erreur est renvoyée :

In [ ]:
! iconv -f ASCII data/ete.latin1.txt

La liste des jeux de caractères est disponible avec l’option `-l` :

In [ ]:
! iconv -l

Demander à décoder un message avec un mauvais jeu de caractères peut amener à représenter les octets avec les mauvais caractères :

In [ ]:
! iconv -f LATIN1 data/ete.utf8.txt

L’option `-t` sert quant à elle à préciser vers quel encodage convertir un fichier :

In [ ]:
! iconv -t ARABIC data/ete.latin1.txt > data/ete.arabic.txt
! iconv -f ARABIC -t ARABIC data/ete.latin1.txt > data/ete.arabic.txt
! cat data/ete.arabic.txt
! iconv -f ARABIC data/ete.arabic.txt

## Comparer des caractères

Les caractères *î* et *î* semblent parfaitement similaires à l’affichage :

In [ ]:
! echo "î"
! echo "î" | iconv -f LATIN1 -t LATIN1

Et pourtant ils proviennent de deux encodages différents : l’UTF-8 et l’ISO-8859-1. Avec la commande `hexdump`, on peut révéler leurs différences sous-jacentes :

In [ ]:
! echo "î" | hexdump -C
! echo "î" | iconv -f LATIN1 | hexdump -C

En ISO-8859-1, quatre octets sont nécessaires pour le coder quand deux suffisent en UTF-8. Si on tente de le convertir en UTF-8, on observe que le système a souhaité afficher deux caractères :

In [ ]:
! echo "î" | iconv -f LATIN1 -t UTF8

En UTF-8, le code hexadécimal du caractère *î* est `c3 ae` quand il est `c3 83 c2 ae` en ISO-8859-1. Que se passe-t-il en réalité ? En fait, quand nous avons affirmé plus haut que quatre octets étaient nécessaires pour coder le caractère *î* en ISO-8859-1, nous avons commis une faute d’appréciation. En vérité, le système a eu besoin de quatre octets pour afficher dans la sortie standard, qui elle est en UTF-8, l’interprétation du caractère selon le jeu ISO-8859-1. Or, le résultat de l’interprétation avait bien été faite sur deux caractères : *Ã®*.

D’où proviennent ces caractères ? Le système les a sans surprise récupérés dans le jeu ISO-8859-1 comme indiqué. Dans ce jeu, le code `c3` correspond au caractère *Ã* et le code `ae` au caractère *®*. Si nous tentons d’afficher en UTF-8 le résultat de la conversion de ces caractères de l’UTF-8 vers l’ISO-8859-1, nous retrouvons bien notre caractère :

In [ ]:
! echo "Ã®" | iconv -f UTF8 -t LATIN1